<a href="https://colab.research.google.com/github/francescoS01/Bayesian-Network/blob/main/bayesian_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### **1. Bayesian node class**

##### **1.1 get and set**:
##### **1.2 value generate**: 
jbnjue

In [1]:
import random
from typing import List

class BayesianNode:
    def __init__(self, node_name, possible_value, cpt, current_value=None, children=[], parents=[]):
        self.node_name = node_name
        self.possible_value = possible_value
        self.children = children
        self.parents = parents
        self.cpt = cpt
        self.current_value = current_value  # Aggiungo l'attributo per memorizzare lo stato corrente
        self.children_update()  # Chiamata al metodo update al momento dell'inizializzazione

    def children_update(self):
        for parent in self.parents:
            parent.set_children(self)

    def set_children(self, new_child: 'BayesianNode'):
        self.children.append(new_child)

    def set_parents(self, parents):
        self.parents = parents

    def set_current_state(self, value): 
        self.current_value = value

    def get_name(self):
        return self.node_name

    def get_parents(self):
        return self.parents
    
    def get_children(self): 
        return self.children
     
    def get_current_state(self): 
        return self.current_value
    
    def get_cpt(self): 
        return self.cpt 
    
    def value_generate(self):
        # create a dictionary with paretns and them value
        parent_value_dict = {}
        for parent in self.parents :
            parent_name = parent.get_name()
            value = parent.get_current_state()
            parent_value_dict[parent_name] = value
        
        # create a dictionary of probability fo self node knowing parents and them values 
        probability_distribution = {}
        for value in self.possible_value:
            new = parent_value_dict.copy()
            new[self.node_name] = value
            for dict in self.cpt: 
                new['prob'] = dict['prob']
                if dict == new:
                    probability_distribution[value] = dict['prob']
        
        # foloowing the probility, extract one value of the self node 
        numero_random = random.random() # 0 -1 
        accumulate = 0
        for key, prob in probability_distribution.items():
            accumulate += prob
            if accumulate >= numero_random:
                self.current_value = key
                return key

### **2. Network node class**


In [2]:
import random
from bayesian_node import *
# Ora puoi utilizzare le funzioni e le classi definite in node.py

from typing import List

class Network:
    def __init__(self, nodes:List[BayesianNode]):
        self.net_nodes = nodes

    def topological_sort(self):
        visited = []
        parents_count = {node: 0 for node in self.net_nodes}

        # InizializzO il conteggio dei genitori per ciascun nodo
        for node in self.net_nodes:
            for child in node.children:
                parents_count[child] += 1

        # TrovO nodi senza genitori
        no_parents = [node for node in self.net_nodes if parents_count[node] == 0]

        # IterO sui nodi senza genitori e visita i loro figli
        while no_parents:
            node = no_parents.pop(0)
            visited.append(node)
            for child in node.children:
                parents_count[child] -= 1
                if parents_count[child] == 0:
                    no_parents.append(child)

        return visited


### **3. Node creation**

In [3]:
# NODE 1: healt node
possible_value = ['good', 'not good']
healt_node_name = 'healt'
current_value = None
parents = []
children = []
cpt = [{healt_node_name:'good', 'prob': 0.7}, 
       {healt_node_name:'not good', 'prob': 0.7}]
healt_node = BayesianNode(healt_node_name, possible_value , cpt, current_value, children, parents)

# NODE 2: stress node 
possibile_value = ['stressed', 'not stressed']
stress_node_name = 'stress'
current_value = None
parents = [healt_node]
children = []
healt_name = healt_node.get_name()
cpt =  [{stress_node_name: 'stressed', healt_node_name: 'good', 'prob':0.2}, 
        {stress_node_name: 'stressed', healt_node_name: 'not good', 'prob':0.7},
        {stress_node_name: 'not stressed', healt_node_name: 'good', 'prob':0.8},
        {stress_node_name: 'not stressed', healt_node_name: 'good', 'prob':0.3}]
stress_node = BayesianNode(stress_node_name, possible_value , cpt, current_value, children, parents)

# NODE 3: sleep node
sleep_possibile_value = ['good', 'not good']
sleep_node_name = 'sleep'
parents = [healt_node, stress_node]
children = []
current_value = None
healt_name = healt_node.get_name()
stress_name = stress_node.get_name()
cpt = [                                                                                                 
    {sleep_node_name: 'good', healt_name: 'good', stress_name: 'stressed', 'prob':0.3},
    {sleep_node_name: 'good', healt_name: 'good', stress_name: 'not stressed', 'prob':0.5},
    {sleep_node_name: 'good', healt_name: 'not good', stress_name: 'stressed', 'prob':0.1}, 
    {sleep_node_name: 'good', healt_name: 'not good', stress_name: 'not stressed', 'prob':0.5},
    {sleep_node_name: 'not good', healt_name: 'good', stress_name: 'stressed', 'prob':0.7},
    {sleep_node_name: 'not good', healt_name: 'good', stress_name: 'not stressed', 'prob':0.5},
    {sleep_node_name: 'not good', healt_name: 'not good', stress_name: 'stressed', 'prob':0.9},
    {sleep_node_name: 'not good', healt_name: 'not good', stress_name: 'not stressed', 'prob':0.5}]
sleep_node = BayesianNode(sleep_node_name, possible_value , cpt, current_value, children, parents)

#### **4. Main**

In [4]:
# ---------------------- NODE TEST ----------------------------
healt_node.set_current_state('not good')
stress_node.set_current_state('not stressed')
z = sleep_node.value_generate()
print(z)



# ------------------------- NET TEST --------------------------------
net = Network([sleep_node, stress_node, healt_node])
order_nodes = net.topological_sort()
for node in order_nodes: 
    print(node.get_name())


good
healt
stress
sleep
